In [12]:
import os
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [13]:
#get DataFrame of all stations' monthly time spent totally full or empty

#hacky method of getting folder names and breaking csvs into four groups
pathName1 = os.getcwd() + '/AvailabilityData'
pathName2 = os.getcwd() + '/AvailabilityData2'
folderNames = os.listdir(pathName1) + os.listdir(pathName2)
folderBatch1 = folderNames[1:int(len(folderNames)/4+1)]
folderBatch2 = folderNames[int(len(folderNames)/4+1):int(len(folderNames)/2+1)]
folderBatch3 = folderNames[int(len(folderNames)/2+1):int(len(folderNames)*3/4+1)]
folderBatch4 = folderNames[int(len(folderNames)*3/4+1):]

# loop through each folder batches to append csv data to a dataframe (even the dirty data, coerce dtype errors later)
dock_availability_df = pd.DataFrame()
for folder in folderBatch1:
    dock_availability_df = dock_availability_df.append(pd.read_csv(pathName1+'/'+ folder
                                                                   +'/bikeshare_nyc_raw.csv',sep='\t', 
                                                                   error_bad_lines = False))
    
dock_availability_df.info()

b'Skipping line 1224513: expected 13 fields, saw 14\n'
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,7,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 1262292: expected 14 fields, saw 15\nSkipping line 1262293: expected 14 fields, saw 15\n'
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 890470: expected 14 fields, saw 26090\n'
b'Skipping line 919921: expected 14 fie

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11356751 entries, 0 to 1511307
Data columns (total 17 columns):
_lat            float64
_long           object
avail_bikes     object
avail_bikes3    float64
avail_docks     object
date            object
dock_id         object
dock_name       object
dock_name6      object
dock_name7      object
hour            int64
in_service      float64
lat3            float64
minute          int64
pm              int64
status_key      object
tot_docks       float64
dtypes: float64(5), int64(3), object(9)
memory usage: 1.5+ GB


In [14]:
#create list of column_names created by ignoring bad line errors
clean_column_names = ['dock_id','dock_name','date','hour','minute','pm','avail_bikes','avail_docks','tot_docks','_lat','_long']
all_columns = dock_availability_df.columns
dirty_columns = []
for column in all_columns:
    if column not in clean_column_names:
        dirty_columns.append(column)
    
#delete the rows that have values for the dirty columns and delete the dirty columns
for i in dirty_columns:
    dock_availability_df = dock_availability_df.drop(dock_availability_df[i].isnull())
    dock_availability_df = dock_availability_df.drop(i, axis=1)

In [15]:
print(dock_availability_df.info())
print(dock_availability_df.count())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11356733 entries, 2 to 1511307
Data columns (total 11 columns):
_lat           float64
_long          object
avail_bikes    object
avail_docks    object
date           object
dock_id        object
dock_name      object
hour           int64
minute         int64
pm             int64
tot_docks      float64
dtypes: float64(2), int64(3), object(6)
memory usage: 1.0+ GB
None
_lat           11356731
_long          11356731
avail_bikes    11356729
avail_docks    11356732
date           11356733
dock_id        11356733
dock_name      11356516
hour           11356733
minute         11356733
pm             11356733
tot_docks      11356732
dtype: int64


In [20]:
#convert columns to correct dtypes -decrease memory usage- and coerce errors
numeric_columns = ['dock_id','hour','minute','avail_bikes','avail_docks','tot_docks','_lat','_long']
dock_availability_df['pm'] = dock_availability_df['pm'].astype('category')
dock_availability_df['dock_name'] = dock_availability_df['dock_name'].astype(str)
dock_availability_df['date'] = pd.to_datetime(dock_availability_df['date'], format = '%y-%m-%d')

for column in numeric_columns:
    dock_availability_df[column] = pd.to_numeric(dock_availability_df[column], errors = 'coerce')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11356733 entries, 2 to 1511307
Data columns (total 11 columns):
_lat           float64
_long          float64
avail_bikes    float64
avail_docks    float64
date           datetime64[ns]
dock_id        float64
dock_name      object
hour           int64
minute         int64
pm             category
tot_docks      float64
dtypes: category(1), datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 963.9+ MB


In [21]:
#drop rows with NaNs
dock_availability_df = dock_availability_df.dropna()
dock_availability_df.head()

,_lat,_long,avail_bikes,avail_docks,date,dock_id,dock_name,hour,minute,pm,tot_docks
2,40.767272,-73.993929,8.0,30.0,2015-03-01,72.0,W 52 St & 11 Ave,1,25,0,38.0
3,40.767272,-73.993929,8.0,30.0,2015-03-01,72.0,W 52 St & 11 Ave,1,35,0,38.0
4,40.767272,-73.993929,8.0,30.0,2015-03-01,72.0,W 52 St & 11 Ave,1,45,0,38.0
5,40.767272,-73.993929,7.0,31.0,2015-03-01,72.0,W 52 St & 11 Ave,1,54,0,38.0
6,40.767272,-73.993929,8.0,31.0,2015-03-01,72.0,W 52 St & 11 Ave,2,2,0,39.0
